In [262]:
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.decomposition import PCA

In [263]:
train = pd.read_csv('data/train.csv')
holidays = pd.read_csv('data/holidays_events.csv')
oil = pd.read_csv('data/oil.csv')
stores = pd.read_csv('data/stores.csv')
transactions = pd.read_csv('data/transactions.csv')

In [264]:
train['sales'].describe()

count    3.000888e+06
mean     3.577757e+02
std      1.101998e+03
min      0.000000e+00
25%      0.000000e+00
50%      1.100000e+01
75%      1.958473e+02
max      1.247170e+05
Name: sales, dtype: float64

In [265]:
dates = train['date'].unique()
families = train['family'].unique()

In [266]:
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [267]:
p = 0.80
dates_train_filt = np.random.random(dates.shape) < p

In [268]:
dates_train_ind = dates[dates_train_filt == True]
dates_test_ind = dates[dates_train_filt == False]
print(dates_train_ind.shape, dates_test_ind.shape)

(1337,) (347,)


In [269]:
dates_train_ind

array(['2013-01-01', '2013-01-02', '2013-01-03', ..., '2017-08-12',
       '2017-08-13', '2017-08-14'], shape=(1337,), dtype=object)

In [270]:
dates_train = train[train['date'].isin(dates_train_ind)]
dates_test = train[train['date'].isin(dates_test_ind)]

In [271]:
dtr_ind = dates_train[['date', 'store_nbr']].value_counts().index

In [272]:
dtr_ind

MultiIndex([('2017-08-14', 54),
            ('2013-01-01',  1),
            ('2013-01-01',  2),
            ('2013-01-01',  3),
            ('2013-01-01',  4),
            ('2013-01-01',  5),
            ('2017-08-14', 38),
            ('2017-08-14', 37),
            ('2017-08-14', 36),
            ('2017-08-14', 35),
            ...
            ('2013-01-01', 16),
            ('2013-01-01', 15),
            ('2013-01-01', 14),
            ('2013-01-01', 13),
            ('2013-01-01', 12),
            ('2013-01-01', 11),
            ('2013-01-01', 10),
            ('2013-01-01',  9),
            ('2013-01-01',  8),
            ('2013-01-01',  7)],
           names=['date', 'store_nbr'], length=72198)

In [273]:
tmp = dtr_ind[0]

In [274]:
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [275]:
# train['date'].to_datetime()
dates_dt = pd.to_datetime(train['date'])
dates_dt_min = dates_dt.min()
dates_days = (dates_dt - dates_dt_min).dt.days

In [276]:
extant_days = dates_days.unique()

In [277]:
dates_days.min(), dates_days.max() + 1

(np.int64(0), np.int64(1688))

In [278]:
day_range = np.arange(0, 1688)

In [279]:
day_range_df = pd.DataFrame(np.arange(0, 1688))

In [280]:
# pd.to_timedelta(day_range_df[day_range_df[0].isin(extant_days) == False], unit = 'days')
day_range_df[day_range_df[0].isin(extant_days) == False]

,0
358,358
723,723
1088,1088
1454,1454


In [281]:
dates_dt_min + dt.timedelta(days = 1454)

Timestamp('2016-12-25 00:00:00')

In [282]:
day_range.shape

(1688,)

In [283]:
families.shape

(33,)

In [284]:
"In addition to families, add oil price and holiday data"

'In addition to families, add oil price and holiday data'

In [285]:
store_nbrs = train['store_nbr'].unique()

In [286]:
store_nbrs - 1

array([ 0,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,  1, 19, 20, 21, 22, 23,
       24, 25, 26, 27, 28,  2, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,  3,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48,  4, 49, 50, 51, 52, 53,  5,
        6,  7,  8])

In [287]:
sales_shape = (1688, 54, 33)
holiday_shape = (1688, 54)
oil_shape = (1688)

In [288]:
sales = np.zeros(dtype = np.float64, shape = sales_shape)

In [289]:
fam_index = pd.DataFrame(data = np.arange(33), index = families)[0]

In [290]:
# for ind, row in train.transpose().items():
#     # print(row)
#     day = (pd.to_datetime(row['date']) - dates_dt_min).days
#     store = row['store_nbr'] - 1
#     fam = fam_index[row['family']]
#     # print(row)
#     # print(row['sales'])
#     sales[day, store, fam] = row['sales']

    

In [291]:
"Name Oil"

'Name Oil'

In [292]:
ind

0

In [293]:
train.index

RangeIndex(start=0, stop=3000888, step=1)

In [294]:
train_txf = train.copy()
train_txf['date'] = (pd.to_datetime(train['date']) - dates_dt_min).dt.days
train_txf['store_nbr'] = train['store_nbr'] - 1
train_txf['family'] = train['family'].apply(fam_index.get)
# print(fam_index[train['family']])


In [295]:
train_txf[['date', 'store_nbr', 'family']].iloc[0]['family']

np.int64(0)

In [296]:
for row in train_txf.itertuples():
    day = row.date
    store = row.store_nbr
    fam = row.family
    sales[day, store, fam] = row.sales

In [297]:
oil_txf = oil.copy()
oil_txf['date'] = pd.to_datetime(oil['date'])
oil_txf['day_num'] = (oil_txf['date'] - dates_dt_min).dt.days

In [298]:
oil_txf['dcoilwtico'].dtype

dtype('float64')

In [299]:
# def get_oil_price(date):
#     if date.day_of_week == 5:
#         return oil_txf
# date = oil_filled['date'].iloc[1]
def get_oil_price(date : dt.datetime):
    # print(date)
    # print(date.day_of_week)
    tgt = date
    if date.day_of_week == 5:
        tgt = date - dt.timedelta(days = 1)
    if date.day_of_week == 6:
        tgt = date - dt.timedelta(days = 2)
    # print(tgt)
    price = oil_txf[oil_txf['date'] == tgt]['dcoilwtico'].iloc[0]
    return price

In [300]:
time_arange = np.arange(1688)
oil_filled = pd.DataFrame(index = time_arange, data = {
    'day_num' : pd.to_timedelta(time_arange, unit = 'days')
})
oil_filled['date'] = oil_filled['day_num'] + dates_dt_min
oil_filled['price'] = oil_filled['date'].apply(get_oil_price)

In [301]:
oil_filled.head(10)

,day_num,date,price
0,0 days,2013-01-01,NaN
1,1 days,2013-01-02,93.14
2,2 days,2013-01-03,92.97
3,3 days,2013-01-04,93.12
4,4 days,2013-01-05,93.12
5,5 days,2013-01-06,93.12
6,6 days,2013-01-07,93.20
7,7 days,2013-01-08,93.21
8,8 days,2013-01-09,93.08
9,9 days,2013-01-10,93.81


In [302]:
# oil_txf.head()
# oil_txf[oil_txf['date'] == 5]
# train_txf[train_txf['date'] == 5]
# oil_txf
# pd.to_datetime(oil['date']).dt.day_of_week.value_counts()
date_ind = pd.to_timedelta(np.arange(1688), unit = 'days') + dates_dt_min

In [303]:
date_ind[date_ind.isin(oil_txf['date']) == False].day_of_week.value_counts()

5    241
6    241
Name: count, dtype: int64

In [304]:
# Now `sales` is a 3-dimensional array holding the sales indexed by day, store, and family

In [305]:
np.where(train['family'] == 'AUTOMOTIVE')[0][0]

np.int64(0)

In [306]:
# 
dates_dt.dt.day_of_week.value_counts()

date
1    431244
0    429462
5    429462
3    427680
2    427680
4    427680
6    427680
Name: count, dtype: int64

In [307]:
# train['holiday'] = train['date', 'store_nbr'].apply

In [308]:
pca = PCA()
sales_pca = pca.fit_transform(sales.reshape((-1, 33)))

In [310]:
pca.explained_variance_ratio_[:5].sum()

np.float64(0.9869853896735724)

In [ ]:
# sales.tofile('data/sales_array.')